In [1]:
import pandas as pd
import numpy as np
#import itertools
import matplotlib.pyplot as plt
from sklearn import preprocessing
from itertools import repeat
df = pd.read_csv("CombinedSequences_ppr.csv")

In [2]:
seq = df['Sequence'].to_numpy()

def kmerise(line,k):  # String and k are inputs
    kmerised_array = []; # Final kmerised list
    for i in line:
        temp = []
        for j in range(0,len(i),k): 
            temp.append(i[j:j+k]) #append substring in each iter 
        kmerised_array.append(temp)
    

    return kmerised_array

In [3]:
df['AccessionID'] = pd.factorize(df['AccessionID'])[0]
df.head()

,AccessionID,Sequence
0,0,ATGAGAGTTCAAAGACCACCCACTCTCTTGTTAGTGTTCTCACTCT...
1,0,ATGATACACTCAGTGTTTCTACTGATGTTCTTGTTAACACCTACAG...
2,0,TATGCCTAACATGTGTAGGATTTTCGCGTCTCTGATTTTGGCACGC...
3,0,TATGCCTAACATGTGTAGGATTTTTGCATCTCTGATTTTGGCACGC...
4,0,TATGCCTAACATGTGTAGGATTTTCGCGTCTCTGATTTTGGCACGC...


In [4]:
df.tail()

,AccessionID,Sequence
2519,2,GACTAAAGATAAAAATTATATACGTATAATTTTTGTCTCTCTAGCT...
2520,2,GACAAAGGTGAAAATTAATATATATATATTAATTTTACTCCTCCTC...
2521,2,GTCATTTGGTAAAATATAATATATATTTTATCACTCTAGCTTCGCT...
2522,2,GACAAAGCTCAAAAATATATATATATATTTTTGTTGCTCCTAGCTT...
2523,2,GACTAAAGATAAAAATTATAGCATTAGTCTATAATTTTATCTCCCT...


In [5]:
k = 1
kmers_list =  kmerise(seq,k)

In [6]:
baseDict = {'A': 1, 'C': 2, 'G': 3, 'T': 4, 'R': 5, 'Y': 6, 'S': 7, 'W': 8, 'K': 9, 'M': 10, 'B': 11, 'D': 12, 'H': 13, 'V': 14,'N': 15}

In [7]:
def vectorizedList(line):  # String and k are inputs
    vectorized_array = []; # Final vectorised list
    for i in line:
        temp = ""
        for j in range(len(i)):
            temp += str(baseDict[i[j]])
        vectorized_array.append(int(temp))
    
    return vectorized_array

In [8]:
Vec = []
len_vec = []
for i in range(len(kmers_list)):
    Vec.append(vectorizedList(kmers_list[i]))
    len_vec.append(len(kmers_list[i]))
#Vec

In [9]:
#f = [[float(i) for i in j] for j in Vec]
#Vec = [[i / sum(j) for i in j] for j in f]

In [10]:
def padding(seq, len_vec):
    anotherseq = seq.copy()
    max_kmerlen = max(len_vec)
    for i in anotherseq:
        diff = max_kmerlen - len(i)
        i.extend(repeat(0, diff))
    
    return anotherseq    

In [11]:
pad = padding(Vec, len_vec)

In [12]:
from keras.layers import Conv2D, Input, MaxPooling2D, Dropout, Flatten, Dense, Activation
from keras.layers import Input, Dropout, Flatten, Dense, Activation
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import precision_score, recall_score, f1_score
import keras

In [13]:
X = np.asarray(pad)#.astype(np.float32)
y = df['AccessionID'].to_numpy()#.astype(np.float32)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X.shape

(2524, 31769)

In [15]:
data_shape = [697,47]
X = X.reshape(X.shape[0],data_shape[0],data_shape[1])

ValueError: cannot reshape array of size 80184956 into shape (2524,697,47)

In [16]:
x_train = x_train.astype('float32') 
x_test = x_test.astype('float32') 
x_train /= 15
x_test /= 15

In [17]:
x_train = x_train.reshape(x_train.shape[0], data_shape[0],data_shape[1], 1) 
x_test = x_test.reshape(x_test.shape[0], data_shape[0],data_shape[1], 1) 

In [18]:
y_train = keras.utils.to_categorical(y_train) 
y_test = keras.utils.to_categorical(y_test) 

In [19]:
print(x_train.shape)
print(x_test.shape)

(1886, 697, 47, 1)
(929, 697, 47, 1)


In [20]:
print(y_train.shape)
print(y_test.shape)

(1886, 4)
(929, 4)


In [21]:
from keras.utils.np_utils import to_categorical
from keras.models import Model ,Sequential

In [22]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5), activation='relu', input_shape=(data_shape[0],data_shape[1],1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [25]:
model.compile(optimizer='adam', loss = "mean_squared_error", metrics = ['accuracy']) 
model.fit(x_train, y_train, epochs = 16)

Epoch 1/16
59/59 [==============================] - 53s 884ms/step - loss: 0.0067 - accuracy: 0.9800
Epoch 2/16
59/59 [==============================] - 53s 896ms/step - loss: 0.0043 - accuracy: 0.9924
Epoch 3/16
59/59 [==============================] - 48s 813ms/step - loss: 0.0042 - accuracy: 0.9892
Epoch 4/16
59/59 [==============================] - 49s 835ms/step - loss: 0.0036 - accuracy: 0.9918
Epoch 5/16
59/59 [==============================] - 51s 857ms/step - loss: 0.0040 - accuracy: 0.9898
Epoch 6/16
59/59 [==============================] - 48s 816ms/step - loss: 0.0028 - accuracy: 0.9937
Epoch 7/16
59/59 [==============================] - 48s 811ms/step - loss: 0.0018 - accuracy: 0.9973
Epoch 8/16
59/59 [==============================] - 48s 810ms/step - loss: 0.0019 - accuracy: 0.9957
Epoch 9/16
59/59 [==============================] - 46s 786ms/step - loss: 0.0021 - accuracy: 0.9945
Epoch 10/16
59/59 [==============================] - 47s 795ms/step - loss: 3.0665e-04 - ac

In [26]:
score = model.evaluate(x_test, y_test, verbose=0) 
print('loss=', score[0])
print('accuracy=', score[1])

loss= 0.028424184769392014
accuracy= 0.9375672936439514


In [27]:
y_pred = model.predict(x_test)
Y_pred_classes = np.argmax(y_pred,axis=1)
y_act = np.argmax(y_test,axis=1)

In [28]:
conf_matrix = np.zeros((4,4))
for i in range(len(y_act)):
    conf_matrix[Y_pred_classes[i]][y_act[i]] += 1
conf_matrix

array([[209.,   6.,   0.,   7.],
       [ 29., 561.,   0.,   3.],
       [  0.,   0.,  69.,   0.],
       [ 10.,   1.,   2.,  32.]])

In [ ]:
#roc auc score
roc_auc_score(y_test, y_pred, multi_class='ovo', average='weighted')

In [ ]:
from sklearn.metrics import roc_curve, auc
# roc curve for classes
fpr = {}
tpr = {}
roc_auc ={}

for i in range(fields):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='MERS vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='SARS 1 vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='SARS-COV 2 vs Rest')
plt.title('MLP_kmer ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.show()

### Sigmoid activation

In [15]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5), activation='sigmoid', input_shape=(data_shape[0],data_shape[1],1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, kernel_size=(3,3), activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(1024, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(4, activation='softmax'))

NameError: name 'Sequential' is not defined

In [ ]:
model.compile(optimizer='adam', loss = "mean_squared_error", metrics = ['accuracy']) 
model.fit(x_train, y_train, epochs = 16)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0) 
print('loss=', score[0])
print('accuracy=', score[1])

### tanh activation

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5), activation='tanh', input_shape=(data_shape[0],data_shape[1],1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, kernel_size=(3,3), activation='tanh'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(1024, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(4, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss = "mean_squared_error", metrics = ['accuracy']) 
model.fit(x_train, y_train, epochs = 16)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0) 
print('loss=', score[0])
print('accuracy=', score[1])

### elu activation

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5), activation='elu', input_shape=(data_shape[0],data_shape[1],1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, kernel_size=(3,3), activation='elu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(1024, activation='elu'))
model.add(Dense(64, activation='elu'))
model.add(Dense(4, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss = "mean_squared_error", metrics = ['accuracy']) 
model.fit(x_train, y_train, epochs = 16)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0) 
print('loss=', score[0])
print('accuracy=', score[1])